## Node 정의

이진 검색트리의 Node는 다음과 같은 멤버 변수를 갖는다
* parent: 노드의 parent node의 레퍼런스, parent가 None이면, root node로 간주한다.
* left  : 노드의 왼쪽 child node의 레퍼런스
* right : 노드의 오른쪽 child node의 레퍼런스
* key   : 검색 키 값

또한, 레드블랙트리를 지원하기 위하여 다음과 같은 멤버 변수를 가질 수 있다.
* color : 블랙=1, 레드=0
* nullNode : 레드블랙트리의 leaf node의 child node는 NULL 이 아니라 특별히 정의된 공백 노드의 레퍼런스이다. nullNode의 값이 1이면, 공백 노드를 의미한다. 공백 노드는 블랙 노드이며, 두 child는 모두 None이다.



In [ ]:
class Node:
    def __init__(self, key, nullNode=0):
        self.parent = None
        self.key = key
        self.left = None    #None or empty leaf node for red black tree
        self.right = None   
        
        #these items are for a red black tree
        self.color = 0      #black(1), red(0)
        self.nullNode = nullNode #if it is 1, it means a Null leaf node of a red black tree
    
    def addChild(self, leftOrRight, childNode):
        
        if leftOrRight == 0: #left child
            self.left = childNode 
            childNode.parent = self
            
        elif leftOrRight == 1: #right child            
            self.right = childNode 
            childNode.parent = self

        else :
            return 1
        return 0
    
    
    def readKey(self):
        return self.key
    
    def writeKey(self, key):
        self.key = key
        
        


## BST의 노드 검색

주어진 node를 root로 하는 이진 검색트리에서 key값을 갖는 노드를 검색하여 리턴한다.
* result = 0 이면, key 값을 갖는 노드를 찾음
* result = 1 이면, key 값을 갖는 노드를 찾을 수 없음

In [ ]:
def binarySearchTreeFindNode(node: Node, key) :
    
    if node == None or key == None:
        return 1, None
    
    if node.key == key:
        result = 0    
    elif node.key < key :
        result, node = binarySearchTreeFindNode(node.right, key)
    else :
        result, node = binarySearchTreeFindNode(node.left, key)
            
    return result, node

 ## BST에서 새로운 노드 추가
 
root와 key 값이 주어지면, 주어진 key 값을 갖는 새로운 노드를 생성하여 root를 참조하여 트리에 새로운 노드를 삽입한다.
treeType은 레드블랙트리를 지원하기 위한 파라메타로, treeType=1이면, 레드블랙트리에 노드를 삽입하기 위해 함수가 호출되었음을 의미한다.

root가 None이면, 트리를 새로 생성하는 것으로 간주하고 root를 생성한 후 종료한다.

root가 None이 아니면, key 값을 참조하여 적절한 leaf를 찾아 노드를 삽입한다.

함수 호출 후, result = 0 이 리턴되면 정상종료된 것을 의미하며, 0 보다 큰 값이 리턴되면 오류가 발생한 것으로 간주한다.

In [ ]:
def _binarySearchTreeInsertNode(parent: Node, key, treeType) -> (int, Node):    
     
    if parent == None or parent.nullNode == 1:  # if nullNode==1, it is a empty leaf node for red black tree
        return 1, None
    
    # to check whether input type of 'parent' is Node
    # if isinstance(parent,Node) == False:
    #    return 2, None    
    
    node=None
    
    if key < parent.key:
        result, node =  _binarySearchTreeInsertNode(parent.left, key, treeType)
        if result == 1:
            node = Node(key)
            parent.addChild(0, node) #add a left child to a parent node
            
    else:
        result, node =  _binarySearchTreeInsertNode(parent.right, key, treeType)
        if result == 1:
            node = Node(key)
            parent.addChild(1, node) #add a right child to a parent node
    
    #if a tree is a red black tree, a leaf node is an empty leaf node, not a node which does not any child node
    if treeType == 1:
        emptyNode_left = Node(0,1)  #key is 0 and nullNode is 1
        emptyNode_left.parent=node
        emptyNode_left.color=1
        emptyNode_right = Node(0,1)
        emptyNode_right.parent=node
        emptyNode_right.color=1
            
        node.left = emptyNode_left
        node.right = emptyNode_right        
            
    return 0, node


def binarySearchTreeInsertNode(root: Node, key, treeType=0) -> (int, Node):  
    
    if root == None: #generate a root
        node = Node(key, 0)  
        if treeType == 1:
            emptyNode_left = Node(0,1) #key is 0 and nullNode is 1
            emptyNode_left.parent=node
            emptyNode_left.color=1
            emptyNode_right = Node(0,1)
            emptyNode_right.parent=node
            emptyNode_right.color=1
            
            node.color = 1  #a root is a black node
            node.left = emptyNode_left
            node.right = emptyNode_right 
            
        return 0, node
    else:
        result, node = _binarySearchTreeInsertNode(root, key, treeType)
        return result, node 


## BST에서 노드 삭제

주어진 키 값을 이용하여 노드를 탐색한 후, 탐색된 노드를 삭제한다.

In [ ]:
def _deleteNode(node):
    if node.left == None and node.right == None:
        return None
    
    elif node.left != None and node.right == None:
        return node.left
    
    elif node.left == None and node.right != None:
        return node.right
    
    else:
        target = node.right 
        while target.left != None:
            parent = target
            target = target.left
        
        #delete target
        node.key = target.key
        
        if target == node.right:
            node.right = target.right
        else:
            parent.right = target.right
        return node 

# it finds a node with removeNode.key and then remove the node from tree with the root    
def binarySearchTreeDeleteNode(root:Node, key):
    
    result, removeNode = binarySearchTreeFindNode(root, key)
    
    if result == 0:
        if removeNode.key == root.key:
            root = _deleteNode(removeNode)
            
        elif removeNode.parent.left != None and removeNode.parent.left.key == removeNode.key :
            removeNode.parent.left = _deleteNode(removeNode)
        
        else:
            removeNode.parent.right = _deleteNode(removeNode)
    
    return result, removeNode, root

## BST Test

In [ ]:
result, root=binarySearchTreeInsertNode(None,2)
result, node=binarySearchTreeInsertNode(root,4)
result, node=binarySearchTreeInsertNode(root,5)
result, node=binarySearchTreeInsertNode(root,1)
print(root)
node = binarySearchTreeFindNode(root, 5)
print(node)
result, remove, root = binarySearchTreeDeleteNode(root, 2)
print(root)

## red black tree for BTS

In [ ]:
##################################################
# redBlackTree 
##################################################

# node를 중심으로 쪽으로 회전한다.
# 함수가 None이 아닌 객체 return 하면, 루트노드가 변경된 것이므로 해당 함수를 호출한
# 해당 함수를 호출한 함수가 이를 적절히 반영해서 처리해야 한다.
# temp = redBlackTreeInsertNodeLeftTurn(node)
# if temp != None: root = temp

def redBlackTreeInsertNodeLeftTurn(node: Node):
    #node를 중심으로 왼쪽으로 회전
    
    if node.parent != None:  #if node is not a root node
        if node.parent.right == node:
            node.parent.right = node.right
        else:
            node.parent.left = node.right
    
    node.right.parent = node.parent
    
    temp = node.right.left
    node.right.left = node
    node.parent = node.right
    
    temp.parent = node
    node.right = temp  
    
    root = None
    if node.parent.parent == None:  #if node.parent becomes a root
        root=node.parent
        
    return root


# node를 중심으로 오른쪽으로 회전한다.
# 함수가 None이 아닌 객체 return 하면, 루트노드가 변경된 것이므로 해당 함수를 호출한
# 해당 함수를 호출한 함수가 이를 적절히 반영해서 처리해야 한다.
# temp = redBlackTreeInsertNodeRightTurn(node)
# if temp != None: root = temp

def redBlackTreeInsertNodeRightTurn(node: Node):
    
    if node.parent != None:   #if node is not a root node,
        if node.parent.right == node:
            node.parent.right = node.left
        else:
            node.parent.left = node.left
    
    #node가 root 노드라 해도 아래 절차는 문제 되지 않는다.        
    node.left.parent = node.parent    
    
    temp = node.left.right
    node.left.right = node
    node.parent = node.left
    
    temp.parent = node
    node.left = temp
    
    root = None
    if node.parent.parent == None:  #if node.parent becomes a root
        root=node.parent
        
    return root


# redBlackTree에 노드 삽입을 위한 함수 SET
#     (1) redBlackTreeInsertNodeCheckColor
#     (2) redBlackTreeGenerateEmptyLeafNode
#     (3) redBlackTreeInsertNode
def redBlackTreeInsertNodeCheckColor(node:Node, root:Node):
    
    if node.parent == None or node == root: #the node is a root
        return 0, root
    
    if node.parent.parent == None: #if the depth of node is 2 (root is of depth 1)
        node.color = 0 # red 
        return 0, root
    
    if node.parent.color == 1: # the parent node of a new node is black
        return 0, root    
    else:        
        if node.parent.parent.left == node.parent:
            sibling = node.parent.parent.right
        else:
            sibling = node.parent.parent.left   

        if sibling.color == 0 : # sibling is red  case 1
            node.parent.color = 1               #black
            sibling.color = 1                   #black
            node.parent.parent.color = 0        #red
            if node.parent.parent.parent == None:
                node.parent.parent.color = 1    #black
            else:
                if node.parent.parent.parent.color == 0: #red
                    result, root = redBlackTreeInsertNodeCheckColor(node.parent.parent, root)
        else:                   # sibling is black
            # 교과서의 경우보다 더 세분화 해서 경우를 나눠야 한다.
            # p2의 오른쪽/왼쪽 child인지 부터 나눠서 구현해야 한다.
            if node.parent.parent.right == node.parent:
                if node.parent.left == node:           #case 2-1
                    temp=redBlackTreeInsertNodeRightTurn(node.parent)                 
                    node = node.right  #case 2-2 상황에서의 node로 설정
                     
                #case 2-2
                temp = redBlackTreeInsertNodeLeftTurn(node.parent.parent)
                node.parent.color, node.parent.left.color = node.parent.left.color, node.parent.color
            
                
            else: #node.parent.parent.left == node.parent 인 경
                if node.parent.right == node:           #case 2-1
                    temp=redBlackTreeInsertNodeLeftTurn(node.parent)                 
                    node = node.left  #case 2-2 상황에서의 node로 설정
                     
                #case 2-2
                temp = redBlackTreeInsertNodeRightTurn(node.parent.parent)
                node.parent.color, node.parent.right.color = node.parent.right.color, node.parent.color
                
            if temp != None:
                root = temp    
    return 1, root

   
def redBlackTreeInsertNode(root: Node, key):
    
    if __name__=="__main__": print("insert node with key %d"%key)
    
    if root == None:
        result, root = binarySearchTreeInsertNode(root, key, 1)
        return 0, None, root
    
    result, node = binarySearchTreeInsertNode(root, key, 1)
    
    if result > 0 :
        return result, None
    
    if node.parent != None : # the node is not a root node

        node.color = 0 #red   
        result, root = redBlackTreeInsertNodeCheckColor(node, root)
    
    return 0, node, root



# 주의: 노드 삭제는 child가 하나 이하인 경우만을 고려한다.
# 함수의 입력변수 node_x는 노드가 삭제되고, 새로 parent에 연결된 노드를 의미한다.
# 특히, 이 함수는 node_x를 기준으로 블랙의 수가 1개 부족한 경우를 처리하기 위해 호출한다.
def redBlackTreeDeleteNodeCheckColor(node_x:Node, root:Node):
    
    if node_x == None or root == None:
        return 1, None
    
    if node_x.parent == None: # 삭제된 노드를 대신하여 새로 연결된 노드가 루트가  경우
        node_x.color = 1
        return 0, node_x
    
    temp = None
    result = 0
    
    parent = node_x.parent
    #get sibling of deleted node
    if parent.right == node_x:
        sibling = parent.left
    else:
        sibling = parent.right
    
    #case1-1: p, s, l, r = 0, 1, 1, 1        
    if parent.color == 0 and sibling.left.color == 1 and sibling.right.color == 1:
        #p is red  and s is black
        parent.color = 1
        sibling.color = 0
        
    #case *-2: (주의) sibling 이 parent의 왼쪽 또는 오른쪽 노드인지를 구분하여 처리해야 한다.
    elif parent.right == sibling and sibling.right.color == 0:
        # parent를 중심으로 왼쪽으로 회전 
        temp = redBlackTreeInsertNodeLeftTurn(parent)
        parent.color, sibling.color = sibling.color, parent.color
        sibling.right.color = 1
    elif parent.left == sibling and sibling.left.color == 0:
        # parent를 중심으로 오른쪽으로 회전
        temp = redBlackTreeInsertNodeRightTurn(parent) 
        parent.color, sibling.color = sibling.color, parent.color
        sibling.left.color = 1 
            
    
    #case *-3: (주의) sibling 이 parent의 왼쪽 또는 오른쪽 노드인지를 구분하여 처리해야 한다.
    # sibling 과 parent 사이에 노드가 회전하면서 끼어든다. 그러므로 회전으로 루트가 변경되지는 않는다.   
    elif parent.right== sibling and sibling.left.color == 0 and sibling.right.color == 1:
        temp = redBlackTreeInsertNodeRightTurn(sibling)
        sibling.parent.color, sibling.color = sibling.color, sibling.parent.color
        # case *-2
        redBlackTreeDeleteNodeCheckColor(node_x, root)
    elif parent.left== sibling and sibling.left.color == 1 and sibling.right.color == 0:
        temp = redBlackTreeInsertNodeLeftTurn(sibling)
        sibling.parent.color, sibling.color = sibling.color, sibling.parent.color
        # case *-2
        redBlackTreeDeleteNodeCheckColor(node_x, root)
        
    
    #case 2-1
    elif sibling.color == 1 and sibling.left.color == 1 and sibling.right.color == 1:
        sibling.color = 0  #red
        result, temp = redBlackTreeDeleteNodeCheckColor(parent)
        
    #case 2-4: (주의) sibling 이 parent의 왼쪽 또는 오른쪽 노드인지를 구분하여 처리해야 한다.    
    elif parent.color == 1 and sibling.color == 0 and sibling.left.color == 1 and sibling.right.color == 1:
        if parent.right == sibling:
            redBlackTreeInsertNodeLeftTurn(parent)
        else:
            redBlackTreeInsertNodeRightTurn(parent)
        parent.color, sibling.color = sibling.color, node.parent.color
        #case 1으로 변환 
        result, temp = redBlackTreeDeleteNodeCheckColor(node_x)
        
    
    if temp != None:
        root = temp
        
    return result, root    

# if this function returns (result > 0, None), it means there is error
# if this functions returns (0, None), it means a root is deleted         
def redBlackTreeDeleteNode(root:Node, key):
    
    result, node = binarySearchTreeFindNode(root, key)
    
    if result > 0 :
        return result, None
    
    # 삭제 노드가 2개의 child 노드를 갖는 경우 :
    # 삭제 노드의 키 값을 오른쪽 서브 트리의 최소 노드의 키 값으로 치환한 후,
    # 오른쪽 서브 트리의 최소 노드를 삭제하는 것으로 문제를 변경하여 처리한다.
    if node.right.nullNode != 1 and node.left.nullNode != 1:
        target = node.right 
        while target.left != None:
            target = target.left
        
        #delete target
        key = target.key
        
        redBlackTreeDeleteNode(root, key)
        
        node.key = target.key
        
        return 0, root
    
    # 삭제 노드가 1개 또는 0개의 child 노드를 갖는 경우 :
    # 삭제 노드가 레드인 경우, child노드와 parent를 연결시키고 종료한다.
    # 이 경우, 삭제 노드가 1개 또는 0개의 child 노드를 갖는 두 경우가  동일하게 처리된다.
    else:
            
        if node.color == 0: 
            #노드의 색이 레드인 경우 이므로, 해당 노드는 root일 수 없다.
            #그러므로, 이 경우  root의 삭제를 고려하지 않아도 된다.       
            if node.right.nullNode == 1:
                node.parent = node.left
            else:
                node.parent = node.right
            return 0, root
                
        else: # 노드의 색이 블랙인 경우
            # node의 두 child 중 하나는 empty leaf node이다.
            # node의 두 child 중 empty leaf node가 아닌 child를 찾는다.
            # 단, node의 두 child가 모두 empty leaf node인 경우도 함께 처리된다.
            if node.right.nullNonde != 1:  #if right child is not empty leaf node,
                child = node.right
            else:
                child = node.left
                
            child.parent = node.parent
            
            # 삭제할 node가 루트인 경우
            if node.parent == None:
                if node.right.nullNode == 1 and node.left.nullNode == 1 :
                    return 0, node
                if node.right.nullNode != 1:
                    node.right.color = 1
                    return 0, node.right    #새로운 root
                else:
                    node.left.color = 1
                    return 0, node.left     #새로운 root
            
            # 삭제할 node가 루트가 아닌 경
            # delete a node and link its child to its parent
            if node.parent.right == node:
                node.parent.right = child
            else:
                node.parent.left = child
                
            # node가 삭제되어 블랙 노드의 수가 1개 부족하게 된다.
            
            #it is needed to check the color of nodes  
            if child.color==0: 
                child.color = 1  # 블랙 노드의 수가 1 증가하게 된다 -> 문제 해결       
            else: 
                #블랙 노드의 수가 1 부족한 문제를 해결해야 한다.
                redBlackTreeDeleteNodeCheckColor(child, root) 